In [1]:
pip install pandas mysql-connector-python sqlalchemy matplotlib seaborn pymysql plotly.express


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from sqlalchemy import create_engine
from urllib.parse import quote_plus
from datetime import datetime, timedelta
import re

In [3]:
# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("All libraries imported successfully!")

All libraries imported successfully!


## DATABASE CONNECTION AND DATA LOADING

In [4]:
def connect_to_database():
    """Connect to MySQL database and load all tables"""
    try:
# Database credentials
        username = 'root'
        password = quote_plus('Kiit@5793')  # URL-encode special characters
        host = 'localhost'
        port = 3306
        database = 'BizIntel360'

# Create SQLAlchemy engine
        engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}")
        
# List of all tables
        tables = [
            "products", "vendors", "customers", "sales_data", "transactions",
            "inventory", "expense_reports", "customer_feedback", "marketing_campaigns"
        ]
        
# Load all tables into dataframes
        dataframes = {}
        for table in tables:
            try:
                df = pd.read_sql(f"SELECT * FROM {table}", engine)
                dataframes[table] = df
                print(f"✓ Loaded {table}: {df.shape[0]} rows, {df.shape[1]} columns")
            except Exception as e:
                print(f"✗ Error loading {table}: {e}")
        
        return dataframes, engine
    
    except Exception as e:
        print(f"Database connection error: {e}")
        return None, None

# Load data
dataframes, engine = connect_to_database()

✓ Loaded products: 270 rows, 4 columns
✓ Loaded vendors: 270 rows, 4 columns
✓ Loaded customers: 270 rows, 4 columns
✓ Loaded sales_data: 270 rows, 6 columns
✓ Loaded transactions: 270 rows, 5 columns
✓ Loaded inventory: 270 rows, 4 columns
✓ Loaded expense_reports: 270 rows, 5 columns
✓ Loaded customer_feedback: 270 rows, 5 columns
✓ Loaded marketing_campaigns: 270 rows, 5 columns


## DATA EXPLORATION AND BASIC INFO

In [5]:
def explore_dataframes(dataframes):
    """Explore the loaded dataframes"""
    print("="*80)
    print("DATA EXPLORATION SUMMARY")
    print("="*80)
    
    for table_name, df in dataframes.items():
        print(f"\n Table: {table_name.upper()}")
        print("-"*50)
        print(f"Shape: {df.shape}")
        print(f"Columns: {list(df.columns)}")
        print(f"Data Types:\n{df.dtypes}")
        print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024:.2f} KB")
        
# Missing Values
        missing = df.isnull().sum()
        if missing.sum() > 0:
            print(f"Missing Values:\n{missing[missing > 0]}")
        else:
            print("No missing values found.")
        
        print(f"\nFirst 5 rows:\n{df.head(5)}")
        print("\n" + "="*80)


if dataframes:
    explore_dataframes(dataframes)
else:
    print("⚠️ No dataframes found to explore.")


DATA EXPLORATION SUMMARY

 Table: PRODUCTS
--------------------------------------------------
Shape: (270, 4)
Columns: ['product_id', 'product_name', 'category', 'price']
Data Types:
product_id        int64
product_name     object
category         object
price           float64
dtype: object
Memory Usage: 39.48 KB
No missing values found.

First 5 rows:
   product_id product_name        category   price
0           1    Product_1     Electronics   22.26
1           2    Product_2        Clothing  130.00
2           3    Product_3       Groceries  370.87
3           4    Product_4  Home & Kitchen   52.60
4           5    Product_5           Books   25.57


 Table: VENDORS
--------------------------------------------------
Shape: (270, 4)
Columns: ['vendor_id', 'vendor_name', 'contact_email', 'location']
Data Types:
vendor_id         int64
vendor_name      object
contact_email    object
location         object
dtype: object
Memory Usage: 60.69 KB
No missing values found.

First 5 rows:
 

## DATA CLEANING FUNCTIONS

### Product Tables

In [6]:
def clean_products_data(df):
    """Clean Products table data"""
    df_clean = df.copy()
    
# Clean product names
    df_clean = df_clean.drop_duplicates()
    df_clean['product_name'] = df_clean['product_name'].str.strip()
    
# Standardize categories names
    df_clean["category"] = df_clean["category"].str.strip().str.title()
    
# Handle price outliers (price > mean + 3*standard deviations from mean)
    price_mean = df_clean['price'].mean()
    price_std = df_clean['price'].std()
    price_threshold = price_mean + 3 * price_std
    
    print(f"Products with price > ${price_threshold:.2f}(outliers):{(df_clean['price']>price_threshold).sum()}")
    
    return df_clean


### Vendors Table

In [7]:
def clean_vendors_data(df):
    """Clean Vendors table data"""
    df_clean = df.copy()
    
# Remove duplicates
    df_clean = df_clean.drop_duplicates()
    
#  Clean vendor names
    df_clean['vendor_name'] = df_clean['vendor_name'].str.strip()
    
# Clean email format
    email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    invalid_emails = ~df_clean['contact_email'].str.match(email_pattern, na = False)
    print(f"Invalid emails found: {invalid_emails.sum()}")    

# Clean location Names
    df_clean['location'] = df_clean['location'].str.strip().str.title()
    
    return df_clean    

### Customers Table

In [8]:
def clean_customers_data(df):
    """Clean customers table"""
    df_clean = df.copy()
    
# Remove Duplicates
    df_clean = df_clean.drop_duplicates()

# Clean Customers Names
    df_clean['customer_name'] = df_clean['customer_name'].str.strip().str.title()

# Covert join_date to datetime
    df_clean['join_date'] = pd.to_datetime(df_clean['join_date'], errors='coerce') 

# Validate email format 
    email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    invaild_emails = ~df_clean['email'].str.match(email_pattern, na = False)
    print(f"Invalid emails found: {invaild_emails.sum()}")
    
    return df_clean

### Sales_Data Table

In [9]:
def clean_sales_data(df):
    """Clean sales data table"""
    df_clean = df.copy()
    
# Remove duplicates
    df_clean = df_clean.drop_duplicates()

# Convert date columns to datetime
    df_clean['sale_date'] = pd.to_datetime(df_clean['sale_date'])
    
# Remove negative quantities and prices
    negative_qty = df_clean['quantity'] <= 0
    negative_price = df_clean['total_price'] <= 0
    
    print(f"Negative quantities: {negative_qty.sum()}")
    print(f"Negative prices: {negative_price.sum()}")
    
    df_clean = df_clean[~(negative_qty | negative_price)]

# Calculate unit price
    df_clean['unit_price'] = df_clean['total_price']/df_clean['quantity']
    
    return df_clean

### Transaction Table

In [10]:
def clean_transaction_data(df):
    """Clean Transaction data table"""
    df_clean = df.copy()
    
# Remove Duplicates
    df_clean = df_clean.drop_duplicates()

# Convert transation_date to datetime
    df_clean['transaction_date'] = pd.to_datetime(df_clean['transaction_date'])

# Standardize payment methods
    df_clean["payment_method"] = df_clean["payment_method"].str.strip().str.title()

# Remove negative transaction amount
    negative_amounts = df_clean['transaction_amount'] <= 0
    print(f"Negative transaction amounts: {negative_amounts.sum()}")
    df_clean = df_clean[~negative_amounts]
    
    return df_clean


### Inventory Table

In [11]:
def clean_inventory_data(df):
    """Clean inventory table"""
    df_clean = df.copy()
    
# Remove duplicates
    df_clean = df_clean.drop_duplicates()
    
# Convert restock_date to datetime
    df_clean['restock_date'] = pd.to_datetime(df_clean['restock_date'])
    
# Remove negative stock quantities
    negative_stock = df_clean['stock_quantity'] < 0
    print(f"Negative stock quantities: {negative_stock.sum()}")
    df_clean = df_clean[~negative_stock]
    
    return df_clean

### Expense Reports

In [12]:
def clean_expense_reports_data(df):
    """Clean expense reports table"""
    df_clean = df.copy()
    
    # Remove duplicates
    df_clean = df_clean.drop_duplicates()
    
    # Convert expense_date to datetime
    df_clean['expense_date'] = pd.to_datetime(df_clean['expense_date'])
    
    # Standardize category names
    df_clean['category'] = df_clean['category'].str.strip().str.title()
    
    # Remove negative amounts
    negative_amounts = df_clean['amount'] <= 0
    print(f"Negative expense amounts: {negative_amounts.sum()}")
    df_clean = df_clean[~negative_amounts]
    
    return df_clean



### Customer Feedback

In [13]:
def clean_customer_feedback_data(df):
    """Clean customer feedback table"""
    if df is None or df.empty:
        print("Customer feedback table is empty or not loaded")
        return pd.DataFrame()
    
    df_clean = df.copy()
    
    # Remove duplicates
    df_clean = df_clean.drop_duplicates()
    
    # Convert feedback_date to datetime if exists
    if 'feedback_date' in df_clean.columns:
        df_clean['feedback_date'] = pd.to_datetime(df_clean['feedback_date'])
    
    # Clean rating values (assuming 1-5 scale)
    if 'rating' in df_clean.columns:
        invalid_ratings = ~df_clean['rating'].between(1, 5)
        print(f"Invalid ratings (not 1-5): {invalid_ratings.sum()}")
        df_clean = df_clean[~invalid_ratings]
    
    return df_clean



### Marketing Campaigns

In [14]:
def clean_marketing_campaigns_data(df):
    """Clean marketing campaigns table"""
    if df is None or df.empty:
        print("Marketing campaigns table is empty or not loaded")
        return pd.DataFrame()
    
    df_clean = df.copy()
    
    # Remove duplicates
    df_clean = df_clean.drop_duplicates()
    
    # Convert date columns to datetime
    date_columns = ['start_date', 'end_date', 'campaign_date']
    for col in date_columns:
        if col in df_clean.columns:
            df_clean[col] = pd.to_datetime(df_clean[col])
    
    # Remove negative budget/spend amounts
    amount_columns = ['budget', 'spend', 'cost']
    for col in amount_columns:
        if col in df_clean.columns:
            negative_amounts = df_clean[col] < 0
            print(f"Negative {col} amounts: {negative_amounts.sum()}")
            df_clean = df_clean[df_clean[col] >= 0]
    
    return df_clean

 ## APPLY DATA CLEANING

In [15]:
def clean_all_data(dataframes):
    """Clean all dataframes"""
    cleaned_data = {}
    cleaning_functions = {
        'products': clean_products_data,
        'vendors': clean_vendors_data,
        'customers': clean_customers_data,
        'sales_data': clean_sales_data,
        'transactions': clean_transaction_data,
        'inventory': clean_inventory_data,
        'expense_reports': clean_expense_reports_data,
        'customer_feedback': clean_customer_feedback_data,
        'marketing_campaigns': clean_marketing_campaigns_data
    }
    
    print("="*80)
    print("DATA CLEANING PROCESS")
    print("="*80)
    
    for table_name, df in dataframes.items():  # ✅ fixed line here
        print(f"\n Cleaning {table_name}...")
        print("-" * 30)
        
        if table_name in cleaning_functions:
            try:
                cleaned_df = cleaning_functions[table_name](df)
                cleaned_data[table_name] = cleaned_df
                print(f"{table_name}: {df.shape[0]} -> {cleaned_df.shape[0]} rows")
            except Exception as e:
                print(f"Error cleaning {table_name}: {e}")
                cleaned_data[table_name] = df
        else:
            cleaned_data[table_name] = df

    return cleaned_data
if dataframes:
    cleaned_dataframes = clean_all_data(dataframes)

DATA CLEANING PROCESS

 Cleaning products...
------------------------------
Products with price > $668.44(outliers):0
products: 270 -> 270 rows

 Cleaning vendors...
------------------------------
Invalid emails found: 0
vendors: 270 -> 270 rows

 Cleaning customers...
------------------------------
Invalid emails found: 0
customers: 270 -> 270 rows

 Cleaning sales_data...
------------------------------
Negative quantities: 0
Negative prices: 0
sales_data: 270 -> 270 rows

 Cleaning transactions...
------------------------------
Negative transaction amounts: 0
transactions: 270 -> 270 rows

 Cleaning inventory...
------------------------------
Negative stock quantities: 0
inventory: 270 -> 270 rows

 Cleaning expense_reports...
------------------------------
Negative expense amounts: 0
expense_reports: 270 -> 270 rows

 Cleaning customer_feedback...
------------------------------
Invalid ratings (not 1-5): 0
customer_feedback: 270 -> 270 rows

 Cleaning marketing_campaigns...
--------

## EXPLORATORY DATA ANALYSIS (EDA)

### EDA for products table

In [16]:
def perform_eda_products(df):
    """EDA for products table"""
    print("="*60)
    print("📊 PRODUCTS TABLE ANALYSIS")
    print("="*60)
### Basic statistics
    print("\n📈 Price Statistics:")
    print(df['price'].describe())
    
### Category analysis
    print("\n📊 Products by Category:")
    category_counts = df['category'].value_counts()
    print(category_counts)

### Visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

### Price distribution
    axes[0,0].hist(df['price'], bins=30, alpha=0.7, color='skyblue')
    axes[0,0].set_title('Price Distribution')
    axes[0,0].set_xlabel('Price ($)')
    axes[0,0].set_ylabel('Frequency')
    
### Price distribution
    axes[0,0].hist(df['price'], bins=30, alpha=0.7, color='skyblue')
    axes[0,0].set_title('Price Distribution')
    axes[0,0].set_xlabel('Price ($)')
    axes[0,0].set_ylabel('Frequency')
    
### Category distribution
    category_counts.plot(kind='bar', ax=axes[0,1], color='lightcoral')
    axes[0,1].set_title('Products by Category')
    axes[0,1].set_xlabel('Category')
    axes[0,1].set_ylabel('Count')
    axes[0,1].tick_params(axis='x', rotation=45)
    
### Price by category boxplot
    df.boxplot(column='price', by='category', ax=axes[1,0])
    axes[1,0].set_title('Price Distribution by Category')
    axes[1,0].set_xlabel('Category')
    axes[1,0].set_ylabel('Price ($)')
    
### Average price by category
    avg_price = df.groupby('category')['price'].mean().sort_values(ascending=False)
    avg_price.plot(kind='bar', ax=axes[1,1], color='gold')
    axes[1,1].set_title('Average Price by Category')
    axes[1,1].set_xlabel('Category')
    axes[1,1].set_ylabel('Average Price ($)')
    axes[1,1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    return {
        'category_distribution': category_counts,
        'price_stats': df['price'].describe(),
        'avg_price_by_category': avg_price
    }

### EDA for sales data table

In [17]:
def perform_eda_sales(df):
    """EDA for sales data table"""
    print("="*60)
    print("💰 SALES DATA ANALYSIS")
    print("="*60)
    
    # Convert sale_date to datetime if not already
    df['sale_date'] = pd.to_datetime(df['sale_date'])
    
    # Basic statistics
    print("\n📈 Sales Statistics:")
    print(df[['quantity', 'total_price', 'unit_price']].describe())
    
    # Time-based analysis
    df['year'] = df['sale_date'].dt.year
    df['month'] = df['sale_date'].dt.month
    df['day_of_week'] = df['sale_date'].dt.day_name()
    
    # Monthly sales
    monthly_sales = df.groupby(['year', 'month']).agg({
        'total_price': 'sum',
        'quantity': 'sum',
        'sale_id': 'count'
    }).reset_index()
    monthly_sales['year_month'] = monthly_sales['year'].astype(str) + '-' + monthly_sales['month'].astype(str).str.zfill(2)
    
    print("\n📅 Monthly Sales Summary:")
    print(monthly_sales[['year_month', 'total_price', 'quantity', 'sale_id']].tail(10))
    
    # Visualizations
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    
    # Sales over time
    monthly_sales.plot(x='year_month', y='total_price', ax=axes[0,0], marker='o')
    axes[0,0].set_title('Monthly Sales Revenue')
    axes[0,0].set_xlabel('Year-Month')
    axes[0,0].set_ylabel('Total Sales ($)')
    axes[0,0].tick_params(axis='x', rotation=45)
    
    # Quantity distribution
    axes[0,1].hist(df['quantity'], bins=20, alpha=0.7, color='lightgreen')
    axes[0,1].set_title('Quantity Distribution')
    axes[0,1].set_xlabel('Quantity')
    axes[0,1].set_ylabel('Frequency')
    
    # Unit price distribution
    axes[0,2].hist(df['unit_price'], bins=30, alpha=0.7, color='orange')
    axes[0,2].set_title('Unit Price Distribution')
    axes[0,2].set_xlabel('Unit Price ($)')
    axes[0,2].set_ylabel('Frequency')
    
    # Sales by day of week
    day_sales = df.groupby('day_of_week')['total_price'].sum()
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    day_sales = day_sales.reindex(day_order)
    day_sales.plot(kind='bar', ax=axes[1,0], color='purple')
    axes[1,0].set_title('Sales by Day of Week')
    axes[1,0].set_xlabel('Day of Week')
    axes[1,0].set_ylabel('Total Sales ($)')
    axes[1,0].tick_params(axis='x', rotation=45)
    
    # Top customers by sales value
    top_customers = df.groupby('customer_id')['total_price'].sum().sort_values(ascending=False).head(10)
    top_customers.plot(kind='bar', ax=axes[1,1], color='red')
    axes[1,1].set_title('Top 10 Customers by Sales Value')
    axes[1,1].set_xlabel('Customer ID')
    axes[1,1].set_ylabel('Total Sales ($)')
    
    # Quantity vs Price scatter
    axes[1,2].scatter(df['quantity'], df['unit_price'], alpha=0.6)
    axes[1,2].set_title('Quantity vs Unit Price')
    axes[1,2].set_xlabel('Quantity')
    axes[1,2].set_ylabel('Unit Price ($)')
    
    plt.tight_layout()
    plt.show()
    
    return {
        'monthly_sales': monthly_sales,
        'day_sales': day_sales,
        'top_customers': top_customers,
        'sales_stats': df[['quantity', 'total_price', 'unit_price']].describe()
    }


### EDA for transactions table

In [18]:
def perform_eda_transactions(df):
    """EDA for transactions table"""
    print("="*60)
    print("💳 TRANSACTIONS ANALYSIS")
    print("="*60)
    
    # Convert transaction_date to datetime
    df['transaction_date'] = pd.to_datetime(df['transaction_date'])
    
    # Payment method analysis
    payment_methods = df['payment_method'].value_counts()
    print("\n💳 Payment Methods Distribution:")
    print(payment_methods)
    
    # Transaction amount statistics
    print("\n💰 Transaction Amount Statistics:")
    print(df['transaction_amount'].describe())
    
    # Payment method vs amount
    payment_stats = df.groupby('payment_method')['transaction_amount'].agg(['count', 'sum', 'mean']).round(2)
    print("\n📊 Payment Method Statistics:")
    print(payment_stats)
    
    # Visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Payment method distribution
    payment_methods.plot(kind='pie', ax=axes[0,0], autopct='%1.1f%%')
    axes[0,0].set_title('Payment Method Distribution')
    axes[0,0].set_ylabel('')
    
    # Transaction amount distribution
    axes[0,1].hist(df['transaction_amount'], bins=30, alpha=0.7, color='teal')
    axes[0,1].set_title('Transaction Amount Distribution')
    axes[0,1].set_xlabel('Transaction Amount ($)')
    axes[0,1].set_ylabel('Frequency')
    
    # Average transaction by payment method
    payment_stats['mean'].plot(kind='bar', ax=axes[1,0], color='coral')
    axes[1,0].set_title('Average Transaction Amount by Payment Method')
    axes[1,0].set_xlabel('Payment Method')
    axes[1,0].set_ylabel('Average Amount ($)')
    axes[1,0].tick_params(axis='x', rotation=45)
    
    # Transaction volume by payment method
    payment_stats['count'].plot(kind='bar', ax=axes[1,1], color='lightblue')
    axes[1,1].set_title('Transaction Volume by Payment Method')
    axes[1,1].set_xlabel('Payment Method')
    axes[1,1].set_ylabel('Number of Transactions')
    axes[1,1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    return {
        'payment_methods': payment_methods,
        'payment_stats': payment_stats,
        'transaction_stats': df['transaction_amount'].describe()
    }

### EDA for inventory table

In [19]:
def perform_eda_inventory(df):
    """EDA for inventory table"""
    print("="*60)
    print("📦 INVENTORY ANALYSIS")
    print("="*60)
    
    # Stock statistics
    print("\n📊 Stock Quantity Statistics:")
    print(df['stock_quantity'].describe())
    
    # Low stock analysis (assuming < 20 is low stock)
    low_stock_threshold = 20
    low_stock_items = df[df['stock_quantity'] < low_stock_threshold]
    print(f"\n⚠️ Low Stock Items (< {low_stock_threshold}): {len(low_stock_items)}")
    
    # Visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Stock quantity distribution
    axes[0,0].hist(df['stock_quantity'], bins=20, alpha=0.7, color='green')
    axes[0,0].set_title('Stock Quantity Distribution')
    axes[0,0].set_xlabel('Stock Quantity')
    axes[0,0].set_ylabel('Frequency')
    axes[0,0].axvline(low_stock_threshold, color='red', linestyle='--', label=f'Low Stock Threshold ({low_stock_threshold})')
    axes[0,0].legend()
    
    # Stock levels categorization
    df['stock_level'] = pd.cut(df['stock_quantity'], 
                              bins=[0, 20, 50, 100, float('inf')], 
                              labels=['Low', 'Medium', 'High', 'Overstocked'])
    stock_levels = df['stock_level'].value_counts()
    stock_levels.plot(kind='bar', ax=axes[0,1], color=['red', 'yellow', 'green', 'blue'])
    axes[0,1].set_title('Stock Level Categories')
    axes[0,1].set_xlabel('Stock Level')
    axes[0,1].set_ylabel('Number of Products')
    axes[0,1].tick_params(axis='x', rotation=45)
    
    # Top 10 products by stock
    top_stock = df.nlargest(10, 'stock_quantity')[['product_id', 'stock_quantity']]
    axes[1,0].barh(range(len(top_stock)), top_stock['stock_quantity'])
    axes[1,0].set_yticks(range(len(top_stock)))
    axes[1,0].set_yticklabels([f'Product {pid}' for pid in top_stock['product_id']])
    axes[1,0].set_title('Top 10 Products by Stock Quantity')
    axes[1,0].set_xlabel('Stock Quantity')
    
    # Bottom 10 products by stock
    bottom_stock = df.nsmallest(10, 'stock_quantity')[['product_id', 'stock_quantity']]
    axes[1,1].barh(range(len(bottom_stock)), bottom_stock['stock_quantity'], color='red')
    axes[1,1].set_yticks(range(len(bottom_stock)))
    axes[1,1].set_yticklabels([f'Product {pid}' for pid in bottom_stock['product_id']])
    axes[1,1].set_title('Bottom 10 Products by Stock Quantity')
    axes[1,1].set_xlabel('Stock Quantity')
    
    plt.tight_layout()
    plt.show()
    
    return {
        'stock_stats': df['stock_quantity'].describe(),
        'low_stock_items': low_stock_items,
        'stock_levels': stock_levels
    }


### EDA for expense reports table

In [20]:
def perform_eda_expenses(df):
    """EDA for expense reports table"""
    print("="*60)
    print("💸 EXPENSE REPORTS ANALYSIS")
    print("="*60)
    
    # Convert expense_date to datetime
    df['expense_date'] = pd.to_datetime(df['expense_date'])
    
    # Expense statistics
    print("\n💰 Expense Amount Statistics:")
    print(df['amount'].describe())
    
    # Category analysis
    category_expenses = df.groupby('category').agg({
        'amount': ['count', 'sum', 'mean']
    }).round(2)
    category_expenses.columns = ['Count', 'Total', 'Average']
    print("\n📊 Expenses by Category:")
    print(category_expenses)
    
    # Monthly expenses
    df['year_month'] = df['expense_date'].dt.to_period('M')
    monthly_expenses = df.groupby('year_month')['amount'].sum()
    
    # Visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Expense amount distribution
    axes[0,0].hist(df['amount'], bins=30, alpha=0.7, color='orange')
    axes[0,0].set_title('Expense Amount Distribution')
    axes[0,0].set_xlabel('Amount ($)')
    axes[0,0].set_ylabel('Frequency')
    
    # Expenses by category
    category_expenses['Total'].plot(kind='bar', ax=axes[0,1], color='purple')
    axes[0,1].set_title('Total Expenses by Category')
    axes[0,1].set_xlabel('Category')
    axes[0,1].set_ylabel('Total Amount ($)')
    axes[0,1].tick_params(axis='x', rotation=45)
    
    # Monthly expense trend
    monthly_expenses.plot(ax=axes[1,0], marker='o', color='red')
    axes[1,0].set_title('Monthly Expense Trend')
    axes[1,0].set_xlabel('Month')
    axes[1,0].set_ylabel('Total Expenses ($)')
    axes[1,0].tick_params(axis='x', rotation=45)
    
    # Average expense by category
    category_expenses['Average'].plot(kind='bar', ax=axes[1,1], color='teal')
    axes[1,1].set_title('Average Expense by Category')
    axes[1,1].set_xlabel('Category')
    axes[1,1].set_ylabel('Average Amount ($)')
    axes[1,1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    return {
        'expense_stats': df['amount'].describe(),
        'category_expenses': category_expenses,
        'monthly_expenses': monthly_expenses
    }

### Perform comprehensive analysis across all tables

In [21]:
def perform_comprehensive_analysis(dataframes):
    """Perform comprehensive analysis across all tables"""
    print("="*80)
    print("🔍 COMPREHENSIVE BUSINESS ANALYSIS")
    print("="*80)
    
    analysis_results = {}
    
    # Individual table analysis
    if 'products' in dataframes and not dataframes['products'].empty:
        analysis_results['products'] = perform_eda_products(dataframes['products'])
    
    if 'sales_data' in dataframes and not dataframes['sales_data'].empty:
        analysis_results['sales'] = perform_eda_sales(dataframes['sales_data'])
    
    if 'transactions' in dataframes and not dataframes['transactions'].empty:
        analysis_results['transactions'] = perform_eda_transactions(dataframes['transactions'])
    
    if 'inventory' in dataframes and not dataframes['inventory'].empty:
        analysis_results['inventory'] = perform_eda_inventory(dataframes['inventory'])
    
    if 'expense_reports' in dataframes and not dataframes['expense_reports'].empty:
        analysis_results['expenses'] = perform_eda_expenses(dataframes['expense_reports'])
    
    return analysis_results


## ADVANCED ANALYTICS AND INSIGHTS

In [40]:
def generate_business_insights(dataframes):
    """Generate key business insights"""
    print("="*80)
    print("💡 KEY BUSINESS INSIGHTS")
    print("="*80)
    
    insights = {}
    
    # Sales Performance Insights
    if 'sales_data' in dataframes:
        sales_df = dataframes['sales_data'].copy()
        sales_df['sale_date'] = pd.to_datetime(sales_df['sale_date'])
        
        # Revenue insights
        total_revenue = sales_df['total_price'].sum()
        avg_order_value = sales_df['total_price'].mean()
        total_orders = len(sales_df)
        
        insights['revenue'] = {
            'total_revenue': total_revenue,
            'average_order_value': avg_order_value,
            'total_orders': total_orders
        }
        
        print(f"\n💰 REVENUE INSIGHTS:")
        print(f"Total Revenue: ${total_revenue:,.2f}")
        print(f"Average Order Value: ${avg_order_value:.2f}")
        print(f"Total Orders: {total_orders:,}")
        
        # Monthly revenue trend
        sales_df['year_month'] = sales_df['sale_date'].dt.to_period('M')
        monthly_revenue = sales_df.groupby('year_month')['total_price'].sum()
        print(f"\n📈 MONTHLY REVENUE TREND:")
        print(monthly_revenue.tail(6))  # Last 6 months
    
    # Product Performance
    if 'products' in dataframes and 'sales_data' in dataframes:
        products_df = dataframes['products'].copy()  # Make a copy to avoid modifying original
        
        # FIXED: Check if category column exists, if not create it
        if 'category' not in products_df.columns:
            # Create categories based on price ranges
            products_df['category'] = pd.cut(products_df['price'], 
                                           bins=[0, 50, 200, 500, float('inf')],
                                           labels=['Budget', 'Mid-range', 'Premium', 'Luxury'])
        
        # Merge sales with products
        product_sales = sales_df.merge(products_df, on='product_id')
        
        # Top selling products
        top_products = product_sales.groupby('product_name')['total_price'].sum().sort_values(ascending=False).head(10)
        print(f"\n🏆 TOP 10 PRODUCTS BY REVENUE:")
        for product, revenue in top_products.items():
            print(f"{product}: ${revenue:,.2f}")
        
        # FIXED: Category performance with error handling
        try:
            category_performance = product_sales.groupby('category')['total_price'].sum().sort_values(ascending=False)
            print(f"\n📦 CATEGORY PERFORMANCE:")
            for category, revenue in category_performance.items():
                if pd.notna(category):  # Skip NaN categories
                    print(f"{category}: ${revenue:,.2f}")
        except Exception as e:
            print(f"\nNote: Category analysis skipped - {str(e)}")
    
    # Customer Insights
    if 'customers' in dataframes and 'sales_data' in dataframes:
        customers_df = dataframes['customers']
        
        # Customer lifetime value
        customer_sales = sales_df.groupby('customer_id')['total_price'].agg(['sum', 'count']).reset_index()
        customer_sales.columns = ['customer_id', 'total_spent', 'order_count']
        
        avg_clv = customer_sales['total_spent'].mean()
        avg_orders_per_customer = customer_sales['order_count'].mean()
        
        print(f"\n👥 CUSTOMER INSIGHTS:")
        print(f"Average Customer Lifetime Value: ${avg_clv:.2f}")
        print(f"Average Orders per Customer: {avg_orders_per_customer:.1f}")
        
        # Top customers
        top_customers = customer_sales.sort_values('total_spent', ascending=False).head(5)
        print(f"\n🌟 TOP 5 CUSTOMERS BY SPENDING:")
        for _, customer in top_customers.iterrows():
            print(f"Customer ID {customer['customer_id']}: ${customer['total_spent']:,.2f} ({customer['order_count']} orders)")
    
    # Inventory Insights
    if 'inventory' in dataframes and 'products' in dataframes:
        inventory_df = dataframes['inventory']
        
        # Low stock alerts
        low_stock = inventory_df[inventory_df['stock_quantity'] < 20]
        print(f"\n⚠️ LOW STOCK ALERTS:")
        print(f"Products with stock < 20: {len(low_stock)}")
        
        # Stock value
        inventory_with_products = inventory_df.merge(products_df, on='product_id')
        inventory_with_products['stock_value'] = inventory_with_products['stock_quantity'] * inventory_with_products['price']
        total_stock_value = inventory_with_products['stock_value'].sum()
        
        print(f"Total Inventory Value: ${total_stock_value:,.2f}")
    
    # Expense Analysis
    if 'expense_reports' in dataframes:
        expenses_df = dataframes['expense_reports'].copy()
        expenses_df['expense_date'] = pd.to_datetime(expenses_df['expense_date'])  # Ensure datetime format
        
        total_expenses = expenses_df['amount'].sum()
        
        try:
            avg_monthly_expenses = expenses_df.groupby(expenses_df['expense_date'].dt.to_period('M'))['amount'].sum().mean()
        except Exception:
            avg_monthly_expenses = total_expenses / 12  # Fallback estimate
        
        print(f"\n💸 EXPENSE INSIGHTS:")
        print(f"Total Expenses: ${total_expenses:,.2f}")
        print(f"Average Monthly Expenses: ${avg_monthly_expenses:,.2f}")
        
        # Expense by category
        if 'category' in expenses_df.columns:
            expense_by_category = expenses_df.groupby('category')['amount'].sum().sort_values(ascending=False)
            print(f"\nExpense by Category:")
            for category, amount in expense_by_category.items():
                print(f"{category}: ${amount:,.2f}")
    
    # Customer Feedback Analysis
    if 'customer_feedback' in dataframes:
        feedback_df = dataframes['customer_feedback']
        
        avg_rating = feedback_df['rating'].mean()
        rating_distribution = feedback_df['rating'].value_counts().sort_index()
        
        print(f"\n⭐ CUSTOMER SATISFACTION:")
        print(f"Average Rating: {avg_rating:.2f}/5")
        print(f"Rating Distribution:")
        for rating, count in rating_distribution.items():
            print(f"{rating} stars: {count} reviews")
    
    return insights

# Run business insights
if cleaned_dataframes:
    business_insights = generate_business_insights(cleaned_dataframes)

💡 KEY BUSINESS INSIGHTS

💰 REVENUE INSIGHTS:
Total Revenue: $385,977.11
Average Order Value: $1429.54
Total Orders: 270

📈 MONTHLY REVENUE TREND:
year_month
2025-01    46496.53
2025-02    30566.81
2025-03    34524.71
2025-04    29253.38
2025-05    23385.82
2025-06    19165.08
Freq: M, Name: total_price, dtype: float64

🏆 TOP 10 PRODUCTS BY REVENUE:
Product_1: $36,563.28
Product_8: $34,487.40
Product_14: $26,390.55
Product_10: $24,408.56
Product_19: $22,622.66
Product_5: $22,210.71
Product_17: $21,116.58
Product_7: $20,796.62
Product_6: $20,384.59
Product_11: $19,500.69

📦 CATEGORY PERFORMANCE:
Electronics: $96,332.64
Clothing: $91,367.39
Groceries: $79,305.97
Books: $61,956.41
Home & Kitchen: $57,014.70

👥 CUSTOMER INSIGHTS:
Average Customer Lifetime Value: $2231.08
Average Orders per Customer: 1.6

🌟 TOP 5 CUSTOMERS BY SPENDING:
Customer ID 59.0: $9,391.81 (5.0 orders)
Customer ID 196.0: $7,554.42 (3.0 orders)
Customer ID 157.0: $7,007.25 (3.0 orders)
Customer ID 227.0: $6,844.91 (5.0

## VISUALIZATION FUNCTIONS

In [38]:
def create_product_analysis_charts(sales_df, products_df):
    """Create product analysis visualizations"""
    
    # Merge sales with products
    product_sales = sales_df.merge(products_df, on='product_id')
    
    # Top products by revenue
    top_products = product_sales.groupby('product_name')['total_price'].sum().sort_values(ascending=False).head(15)
    
    # Check if category column exists, if not create a simple categorization or skip category analysis
    if 'category' not in products_df.columns:
        # Create a simple category based on price ranges or use product name patterns
        # Option 1: Create categories based on price ranges
        products_df['category'] = pd.cut(products_df['price'], 
                                       bins=[0, 50, 200, 500, float('inf')],
                                       labels=['Budget', 'Mid-range', 'Premium', 'Luxury'])
        
        # Re-merge with updated products_df
        product_sales = sales_df.merge(products_df, on='product_id')
    
    # Category performance
    category_performance = product_sales.groupby('category').agg({
        'total_price': 'sum',
        'quantity': 'sum'
    }).reset_index()
    
    # FIXED: Proper subplot specification for pie chart
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Top 15 Products by Revenue', 'Category Performance', 
                       'Price Distribution by Category', 'Quantity vs Revenue'),
        specs=[[{"secondary_y": False}, {"type": "domain"}],  # domain type for pie chart
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # Top Products
    fig.add_trace(
        go.Bar(y=top_products.index, x=top_products.values, orientation='h', name='Revenue'),
        row=1, col=1
    )
    
    # Category Performance - PIE CHART
    fig.add_trace(
        go.Pie(labels=category_performance['category'], values=category_performance['total_price'], name='Category'),
        row=1, col=2
    )
    
    # Price Distribution by Category
    for category in products_df['category'].unique():
        if pd.notna(category):  # Skip NaN categories
            category_prices = products_df[products_df['category'] == category]['price']
            fig.add_trace(
                go.Box(y=category_prices, name=str(category)),
                row=2, col=1
            )
    
    # Quantity vs Revenue Scatter
    fig.add_trace(
        go.Scatter(x=product_sales['quantity'], y=product_sales['total_price'], 
                  mode='markers', name='Sales'),
        row=2, col=2
    )
    
    fig.update_layout(height=1000, title_text="Product Analysis Dashboard")
    fig.show()

## GENERATE VISUALIZATIONS

In [36]:
pip install nbformat>=4.2.0

Note: you may need to restart the kernel to use updated packages.


In [39]:
def generate_all_visualizations(dataframes):
    """Generate all visualization dashboards"""
    print("="*80)
    print("📊 GENERATING VISUALIZATIONS")
    print("="*80)
    
    # Initialize sales_df as None
    sales_df = None
    
    try:
        if 'sales_data' in dataframes:
            sales_df = dataframes['sales_data'].copy()
            sales_df['sale_date'] = pd.to_datetime(sales_df['sale_date'])
            
            print("Creating Revenue Dashboard...")
            create_revenue_dashboard(sales_df)
            
        # Check if sales_df exists before using it in other functions
        if sales_df is not None and 'products' in dataframes:
            print("Creating Product Analysis Charts...")
            create_product_analysis_charts(sales_df, dataframes['products'])
            
        if sales_df is not None and 'customers' in dataframes:
            print("Creating Customer Insights Charts...")
            create_customer_insights_charts(sales_df, dataframes['customers'])
    
    except Exception as e:
        print(f"Error generating visualizations: {e}")
        import traceback
        traceback.print_exc()

# Run visualizations
if cleaned_dataframes:
    generate_all_visualizations(cleaned_dataframes)

📊 GENERATING VISUALIZATIONS
Creating Revenue Dashboard...


Creating Product Analysis Charts...


Creating Customer Insights Charts...
Error generating visualizations: Trace type 'pie' is not compatible with subplot type 'xy'
at grid position (1, 1)

See the docstring for the specs argument to plotly.subplots.make_subplots
for more information on subplot types


Traceback (most recent call last):
  File "C:\Users\shrut\AppData\Local\Temp\ipykernel_25436\1793648353.py", line 25, in generate_all_visualizations
    create_customer_insights_charts(sales_df, dataframes['customers'])
  File "C:\Users\shrut\AppData\Local\Temp\ipykernel_25436\2805648807.py", line 132, in create_customer_insights_charts
    fig.add_trace(
  File "c:\Users\shrut\.conda\envs\nltk_env\lib\site-packages\plotly\graph_objs\_figure.py", line 344, in add_trace
    return super().add_trace(trace, row, col, secondary_y, exclude_empty_subplots)
  File "c:\Users\shrut\.conda\envs\nltk_env\lib\site-packages\plotly\basedatatypes.py", line 2129, in add_trace
    return self.add_traces(
  File "c:\Users\shrut\.conda\envs\nltk_env\lib\site-packages\plotly\graph_objs\_figure.py", line 422, in add_traces
    return super().add_traces(
  File "c:\Users\shrut\.conda\envs\nltk_env\lib\site-packages\plotly\basedatatypes.py", line 2259, in add_traces
    self._set_trace_grid_position(trace, r

## EXPORT RESULTS

In [26]:
import os

def export_analysis_results(dataframes, insights, filename_prefix="bizintel360_analysis"):
    """Export analysis results to files"""
    print("="*80)
    print("💾 EXPORTING ANALYSIS RESULTS")
    print("="*80)

    export_folder = r"E:\BizIntel360 A Data-Driven Financial Performance Dashboard\analysis_dataset"
    os.makedirs(export_folder, exist_ok=True)  # Ensure folder exists

    try:
        # Export cleaned data to CSV
        for table_name, df in dataframes.items():
            filename = os.path.join(export_folder, f"{filename_prefix}_{table_name}_cleaned.csv")
            df.to_csv(filename, index=False)
            print(f"✓ Exported {table_name} to {filename}")
        
        # Export insights to text file
        insights_filename = os.path.join(export_folder, f"{filename_prefix}_insights.txt")
        with open(insights_filename, 'w') as f:
            f.write("BIZINTEL360 BUSINESS INSIGHTS REPORT\n")
            f.write("="*50 + "\n\n")
            
            if 'revenue' in insights:
                f.write("REVENUE INSIGHTS:\n")
                f.write(f"Total Revenue: ${insights['revenue']['total_revenue']:,.2f}\n")
                f.write(f"Average Order Value: ${insights['revenue']['average_order_value']:.2f}\n")
                f.write(f"Total Orders: {insights['revenue']['total_orders']:,}\n\n")
        
        print(f"✓ Exported insights to {insights_filename}")
        
    except Exception as e:
        print(f"❌ Error exporting results: {e}")

# Export results
if cleaned_dataframes and 'business_insights' in locals():
    export_analysis_results(cleaned_dataframes, business_insights)

print("\n" + "="*80)
print("🎉 DATA ANALYSIS COMPLETE!")
print("="*80)


💾 EXPORTING ANALYSIS RESULTS
✓ Exported products to E:\BizIntel360 A Data-Driven Financial Performance Dashboard\analysis_dataset\bizintel360_analysis_products_cleaned.csv
✓ Exported vendors to E:\BizIntel360 A Data-Driven Financial Performance Dashboard\analysis_dataset\bizintel360_analysis_vendors_cleaned.csv
✓ Exported customers to E:\BizIntel360 A Data-Driven Financial Performance Dashboard\analysis_dataset\bizintel360_analysis_customers_cleaned.csv
✓ Exported sales_data to E:\BizIntel360 A Data-Driven Financial Performance Dashboard\analysis_dataset\bizintel360_analysis_sales_data_cleaned.csv
✓ Exported transactions to E:\BizIntel360 A Data-Driven Financial Performance Dashboard\analysis_dataset\bizintel360_analysis_transactions_cleaned.csv
✓ Exported inventory to E:\BizIntel360 A Data-Driven Financial Performance Dashboard\analysis_dataset\bizintel360_analysis_inventory_cleaned.csv
✓ Exported expense_reports to E:\BizIntel360 A Data-Driven Financial Performance Dashboard\analysis